![cyber_photo](cyber_photo.jpg)

Cyber threats are a growing concern for organizations worldwide. These threats take many forms, including malware, phishing, and denial-of-service (DOS) attacks, compromising sensitive information and disrupting operations. The increasing sophistication and frequency of these attacks make it imperative for organizations to adopt advanced security measures. Traditional threat detection methods often fall short due to their inability to adapt to new and evolving threats. This is where deep learning models come into play.

Deep learning models can analyze vast amounts of data and identify patterns that may not be immediately obvious to human analysts. By leveraging these models, organizations can proactively detect and mitigate cyber threats, safeguarding their sensitive information and ensuring operational continuity.

As a cybersecurity analyst, you identify and mitigate these threats. In this project, you will design and implement a deep learning model to detect cyber threats. The BETH dataset simulates real-world logs, providing a rich source of information for training and testing your model. The data has already undergone preprocessing, and we have a target label, `sus_label`, indicating whether an event is malicious (1) or benign (0).

By successfully developing this model, you will contribute to enhancing cybersecurity measures and protecting organizations from potentially devastating cyber attacks.


### The Data

| Column     | Description              |
|------------|--------------------------|
|`processId`|The unique identifier for the process that generated the event - int64 |
|`threadId`|ID for the thread spawning the log - int64|
|`parentProcessId`|Label for the process spawning this log - int64|
|`userId`|ID of user spawning the log|Numerical - int64|
|`mountNamespace`|Mounting restrictions the process log works within - int64|
|`argsNum`|Number of arguments passed to the event - int64|
|`returnValue`|Value returned from the event log (usually 0) - int64|
|`sus_label`|Binary label as suspicous event (1 is suspicious, 0 is not) - int64|

More information on the dataset: [BETH dataset](accreditation.md)

In [4]:
# Make sure to run this cell to use torchmetrics. If you cannot use pip install to install the torchmetrics, you can use sklearn.
!pip install torchmetrics

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 56.8 MB/s eta 0:00:00


In [25]:
# Import required libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as functional
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torchmetrics import Accuracy
# from sklearn.metrics import accuracy_score  # uncomment to use sklearn

In [26]:
# Load preprocessed data
train_df = pd.read_csv('labelled_train.csv')
test_df = pd.read_csv('labelled_test.csv')
val_df = pd.read_csv('labelled_validation.csv')

# View the first 5 rows of training set
train_df.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue,sus_label
0,381,7337,1,100,4026532231,5,0,1
1,381,7337,1,100,4026532231,1,0,1
2,381,7337,1,100,4026532231,0,0,1
3,7347,7347,7341,0,4026531840,2,-2,1
4,7347,7347,7341,0,4026531840,4,0,1


In [27]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763144 entries, 0 to 763143
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   processId        763144 non-null  int64
 1   threadId         763144 non-null  int64
 2   parentProcessId  763144 non-null  int64
 3   userId           763144 non-null  int64
 4   mountNamespace   763144 non-null  int64
 5   argsNum          763144 non-null  int64
 6   returnValue      763144 non-null  int64
 7   sus_label        763144 non-null  int64
dtypes: int64(8)
memory usage: 46.6 MB


In [28]:
train_df.shape

(763144, 8)

In [29]:
# Start coding here
# Use as many cells as you need

In [30]:
train_features = train_df.drop("sus_label", axis=1)
train_features.head()

,processId,threadId,parentProcessId,userId,mountNamespace,argsNum,returnValue
0,381,7337,1,100,4026532231,5,0
1,381,7337,1,100,4026532231,1,0
2,381,7337,1,100,4026532231,0,0
3,7347,7347,7341,0,4026531840,2,-2
4,7347,7347,7341,0,4026531840,4,0


In [31]:
train_labels = train_df["sus_label"]
train_labels.head()

0    1
1    1
2    1
3    1
4    1
Name: sus_label, dtype: int64

In [32]:
test_features = test_df.drop("sus_label", axis=1)
test_labels = test_df["sus_label"]

In [33]:
val_features = val_df.drop("sus_label", axis=1)
val_labels = val_df["sus_label"]

In [34]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_features)
scaled_test = scaler.transform(test_features)
scaled_val = scaler.transform(val_features)

In [35]:
train_features_tensor = torch.tensor(scaled_train).float()
train_label_tensor = torch.tensor(train_labels).reshape(-1,1)
test_features_tensor = torch.tensor(scaled_test).float()
test_label_tensor = torch.tensor(test_labels).reshape(-1,1)
val_features_tensor = torch.tensor(scaled_val).float()
val_label_tensor = torch.tensor(val_labels).reshape(-1,1)

In [36]:
model = nn.Sequential(
    nn.Linear(train_features.shape[1],128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64,1),
    nn.ReLU(),
    nn.Sigmoid()
)

In [37]:
loss_fn = nn.CrossEntropyLoss()  # binary cross entropy
optimizer = optim.SGD(model.parameters(), lr=1e-3, weight_decay=1e-4)

In [39]:
epochs = range(0, 25)
for epoch in epochs: 
    model.train()
    optimizer.zero_grad()
    output = model(train_features_tensor)
    loss = loss_fn(output, train_label_tensor.float())  # Convert labels to float
    loss.backward()
    optimizer.step()

In [40]:
model.eval()
with torch.no_grad():
    train_lables_predicted = model(train_features_tensor).round()
    test_lables_predicted = model(test_features_tensor).round()
    val_lables_predicted = model(val_features_tensor).round()

In [41]:
accuracy = Accuracy(task='binary')

In [42]:
train_accuracy = accuracy(train_lables_predicted, train_label_tensor).item()
test_accuracy = accuracy(test_lables_predicted, test_label_tensor).item()
val_accuracy = accuracy(val_label_tensor, val_label_tensor).item()

In [44]:
print(f'Train accuracy {train_accuracy}')
print(f'Test accuracy {test_accuracy}')
print(f'Validation accuracy {val_accuracy}')

Train accuracy 0.2826622426509857
Test accuracy 0.06282578408718109
Validation accuracy 1.0
